#1.IMPORT LIBRARIES

In [12]:
import pandas as pd


#2.LOAD AND EXPLORE DATA

In [ ]:
df=pd.read_csv("D:/titanic-survival-prediction/tested.csv")
df.head()
df.info()
df.describe()


#3. DATA PREPROCESSING

# fill missingAge with median
df['Age'].fillna(df['Age'].median())

# Fill missing Embarked with mode
df['Embarked'].fillna(df['Embarked'].mode()[0])

# Drop 'Cabin' due to too many missing values (optional)
df.drop(['Cabin', 'Ticket', 'Name', 'PassengerId'], axis=1)

df['Sex'] = LabelEncoder().fit_transform(df['Sex'])
df = pd.get_dummies(df, columns=['Embarked'], drop_first=True)

scaler = StandardScaler()
df[['Age', 'Fare']] = scaler.fit_transform(df[['Age', 'Fare']])

X = df.drop('Survived', axis=1)
y = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# modeling
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d')
plt.show()

import  joblib
joblib.dump(model, 'models/titanic_model.pkl')






#4.CREATE INTERACTIVE DASHBOARD USING ipywidgets

In [ ]:
import pandas as pd
import seaborn as sns
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import display,Markdown

#load dataset
df= pd.read_csv('D:/titanic-survival-prediction/tested.csv')

#display a title
display(Markdown("##Titanic Prediction Dashboard"))

#step1.define your plot function
def show_pie_chart(feature,pclass,sex):
    filtered=df[(df['Pclass']==pclass)&(df['Sex']==sex)]
    plt.figure(figsize=(12,5))
    sns.set(style="whitegrid")

#prepare survival count (always include 0 and 1 keys)
    survived_count=filtered['Survived'].value_counts().sort_index().to_dict()
    survived_count[0]=survived_count.get(0,0)
    survived_count[1]=survived_count.get(1,0)
    
     
    #pie chart of selected feature
    plt.figure(figsize=(10,4))

    #left:pie chart
    plt.subplot(1,2,1)
    if not filtered.empty:

      filtered[feature].value_counts().plot.pie(autopct='%1.1f%%',startangle=90,colors=sns.color_palette('pastel'))
    else:
       #dummy pie chart when no data
       plt.pie([1],labels=['No Data'],colors=['lightblue'])
    plt.title(f"{feature} Distribution")
    plt.ylabel("")
    
    #right:bar chart of survial
    
    plt.subplot(1,2,2)
    survived_count=filtered['Survived'].value_counts().sort_index()
    labels = ['Did Not Survive', 'Survived']
    values = [survived_count.get(0, 0), survived_count.get(1, 0)]

   #plotting
    plt.plot(survived_count.index,survived_count.values,marker='o',color='lightblue')
    plt.scatter(survived_count.index,survived_count.values,color='red',s=100)
    
    sns.barplot(x=labels,y=values,legend=True)
    plt.xlabel('Outcomes')
    plt.ylabel('Number of passengers')
    plt.title("Survived Count")
    plt.tight_layout()
    plt.show()
    
#step2. create widgets
feature_dropdown=widgets.Dropdown(options=['Survived','Pclass','Sex'], description='Feature:')
pclass_dropdown=widgets.Dropdown(options=[1,2,3],description='Pclass:')
sex_dropdown=widgets.Dropdown(options=['Male','Female'],value='Male',description='Sex:')

#step3. interactive output
ui=widgets.VBox([feature_dropdown,pclass_dropdown,sex_dropdown])
out=widgets.interactive_output(show_pie_chart,{
    'feature':feature_dropdown,
    'pclass':pclass_dropdown,
    'sex':sex_dropdown
})

#step4. display the dashboard
display(ui,out)



    
